<a href="https://colab.research.google.com/github/Sushmitha-Katti/Artshaala--Frontend/blob/master/S4-FaceRecognitionPart2/Session4_quantised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install torch==1.5.0 torchvision==0.6.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 603.7MB 29kB/s 
     |████████████████████████████████| 6.5MB 5.8MB/s 
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101
  Found existing installation: torchvision 0.7.0+cu101
    Uninstalling torchvision-0.7.0+cu101:
      Successfully uninstalled torchvision-0.7.0+cu101


In [1]:
!git clone https://github.com/Sushmitha-Katti/PyTNet
import sys
sys.path.append("PyTNet")

Cloning into 'PyTNet'...
remote: Enumerating objects: 160, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 160 (delta 86), reused 99 (delta 40), pack-reused 0
Receiving objects: 100% (160/160), 42.16 KiB | 553.00 KiB/s, done.
Resolving deltas: 100% (86/86), done.


In [3]:
!git clone https://github.com/timesler/facenet-pytorch.git
!mv ./facenet-pytorch/models .
from models.inception_resnet_v1 import InceptionResnetV1

Cloning into 'facenet-pytorch'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 1188 (delta 19), reused 10 (delta 2), pack-reused 1143
Receiving objects: 100% (1188/1188), 22.80 MiB | 15.15 MiB/s, done.
Resolving deltas: 100% (569/569), done.


In [4]:
from zipfile import ZipFile 
  
# specifying the zip file name 
file_name = "/content/drive/My Drive/lfw_funneled_modified.zip"
  
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
    # printing all the contents of the zip file 
    zip.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!') 

File Name                                             Modified             Size
lfw_funneled/Elon_Musk/Elon_Musk_0009.jpg      2020-09-07 11:49:42         8954
lfw_funneled/Elon_Musk/Elon_Musk_0010.jpg      2020-09-07 11:49:42         9761
lfw_funneled/Elon_Musk/Elon_Musk_0004.jpg      2020-09-07 11:49:42         8383
lfw_funneled/Elon_Musk/Elon_Musk_0008.jpg      2020-09-07 11:49:42        10798
lfw_funneled/Elon_Musk/Elon_Musk_0007.jpg      2020-09-07 11:49:42         7565
lfw_funneled/Elon_Musk/Elon_Musk_0001.jpg      2020-09-07 11:49:42         8675
lfw_funneled/Elon_Musk/Elon_Musk_0002.jpg      2020-09-07 11:49:42         7588
lfw_funneled/Elon_Musk/Elon_Musk_0003.jpg      2020-09-07 11:49:42         8160
lfw_funneled/Elon_Musk/Elon_Musk_0005.jpg      2020-09-07 11:49:42         9085
lfw_funneled/Elon_Musk/Elon_Musk_0006.jpg      2020-09-07 11:49:42         8656
lfw_funneled/George_Clooney/George_Clooney_0002.jpg 2020-09-07 11:49:42        22003
lfw_funneled/George_Clooney/George_

In [6]:
# Total No Of Classes
!ls '/content/lfw_funneled'

Abdullah_Gul		   Jackie_Chan		      Mohammed_Al-Douri
Adrien_Brody		   Jack_Straw		      Monica_Gellar
Alejandro_Toledo	   Jacques_Chirac	      Muhammad_Ali
Alvaro_Uribe		   Jacques_Rogge	      Nancy_Pelosi
Amelie_Mauresmo		   James_Blake		      Naomi_Watts
Andre_Agassi		   James_Kelly		      Nestor_Kirchner
Andy_Roddick		   Jason_Kidd		      Nicanor_Duarte_Frutos
Angelina_Jolie		   Javier_Solana	      Nicole_Kidman
Anna_Kournikova		   Jean_Charest		      Norah_Jones
Ann_Veneman		   Jean_Chretien	      Paradorn_Srichaphan
APJ_Abdul_Kalam		   Jean-David_Levitte	      Paul_Bremer
Ariel_Sharon		   Jeb_Bush		      Paul_Burrell
Ari_Fleischer		   Jennifer_Aniston	      Paul_ONeill
Arnold_Schwarzenegger	   Jennifer_Capriati	      Paul_Wolfowitz
Atal_Bihari_Vajpayee	   Jennifer_Garner	      Pervez_Musharraf
Barack_Obama		   Jennifer_Lopez	      Pete_Sampras
Bill_Clinton		   Jeong_Se-hyun	      Pheobe_Buffay
Bill_Frist		   Jeremy_Greenstock	      Pierce_Brosnan
Bill_Gates		   Jesse_Jackson	    

In [7]:
from torch import nn, optim, as_tensor
import torchvision
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.nn.init import *
from torchvision import transforms, utils, datasets, models
from models.inception_resnet_v1 import InceptionResnetV1
import cv2
from PIL import Image
from pdb import set_trace
import time
import copy
from pathlib import Path
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage import io, transform
from tqdm import trange, tqdm
import csv
import glob
import dlib
import pandas as pd
import numpy as np
from torchsummary import summary

In [8]:
train_transforms =transforms.Compose([
        transforms.Resize(size=(224, 224),interpolation=2),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])   # Normalized value 
    ])

dir = '/content/lfw_funneled/'
dataset = torchvision.datasets.ImageFolder(dir,transform = train_transforms)

In [9]:
def load(trainset,testset,seed=1,batch_size=128,num_workers=4,pin_memory=True):
	

	#Get the Train and Test Set
	# trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
	# testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)


	SEED = 1

	# CUDA?
	cuda = torch.cuda.is_available()


	# For reproducibility
	torch.manual_seed(SEED)

	if cuda:
			torch.cuda.manual_seed(SEED)

	# dataloader arguments - something you'll fetch these from cmdprmt
	dataloader_args = dict( batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory) if cuda else dict(shuffle=True, batch_size=64)

	trainloader = torch.utils.data.DataLoader(trainset, **dataloader_args)
	testloader = torch.utils.data.DataLoader(testset, **dataloader_args)


	return  trainloader, testloader

In [10]:
from torch.utils.data import Dataset, random_split
train_len = len(dataset)*70//100
test_len = len(dataset) - train_len 
train_set, val_set = random_split(dataset, [train_len, test_len])


In [11]:
import train_test_loader
trainloader, testloader = load(train_set,val_set,batch_size=32)
num_classes = len(dataset.classes)

In [12]:
device = "cuda"
model = InceptionResnetV1(pretrained='vggface2', classify=True, num_classes = num_classes).to(device)
model.logits.out_features = num_classes


In [13]:

total_child = len(list(model.children()))
ct = 0
print("\nFreezed layer except below childs/layers \n")

for child in model.children():
  ct += 1
  if ct < total_child - 5:
    for param in child.parameters():
        param.requires_grad = False
  else:
    print(child)


Freezed layer except below childs/layers 

Block8(
  (branch0): BasicConv2d(
    (conv): Conv2d(1792, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (branch1): Sequential(
    (0): BasicConv2d(
      (conv): Conv2d(1792, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (1): BasicConv2d(
      (conv): Conv2d(192, 192, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1), bias=False)
      (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (2): BasicConv2d(
      (conv): Conv2d(192, 192, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), bias=False)
      (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
  )
  (conv2d

In [14]:
for param in model.conv2d_3b.parameters():
  a = 0 #do nothing
print("requires_grad value set to early layer:",param.requires_grad)

requires_grad value set to early layer: False


In [15]:
import torch

#Training & Testing Loops
from tqdm.notebook import tqdm
from tqdm import tqdm_notebook

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, criterion, epoch,scheduler = False):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    #loss = F.nll_loss(y_pred, target)
    loss = criterion(y_pred, target)
    

    # Backpropagation
    loss.backward()
    optimizer.step()
    if(scheduler):
      scheduler.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(f'Loss={loss.item():0.2f} Acc={100*correct/processed:0.2f}')
  train_losses.append(loss)
  train_acc.append(100*correct/processed)
  return train_losses,train_acc


def test(model, device, criterion, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in tqdm(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
	
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))
    return test_losses,test_acc



test_loss = []
test_acc = []
train_loss = []
train_loss = []
LR = []
train_scheduler = False

def train_model(model,device,trainloader,testloader,optimizer,criterion,EPOCHS,scheduler = False,batch_scheduler = False ,best_acc = 0,path = "/content/gdrive/My Drive/API/bestmodel.pt"):
  for epoch in range(EPOCHS):
      print("EPOCH:", epoch+1,'LR:',optimizer.param_groups[0]['lr'])
      LR.append(optimizer.param_groups[0]['lr'])
      train_scheduler = False

      if(batch_scheduler):
        train_scheduler = scheduler
      train_loss, train_acc = train(model, device, trainloader, optimizer, criterion, epoch,train_scheduler)
      
      test_loss , test_acc = test(model, device, criterion, testloader)
      if(scheduler and not batch_scheduler and not isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau)): 
        scheduler.step()

      elif(scheduler and not batch_scheduler and  isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau)):
        scheduler.step(test_loss[-1])
      
      
      if(test_acc[-1]>best_acc):
        print("accuracy increased, Saving model....")
        best_acc = test_acc[-1]
        torch.save({
              'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': test_loss[-1],
              }, path)


In [16]:
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR,StepLR,MultiStepLR, ReduceLROnPlateau

optimizer = optim.SGD(model.parameters(), lr=0.01,momentum=0.9, nesterov=True) 
criterion = nn.CrossEntropyLoss()
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

In [17]:
train_model(model,device,trainloader,testloader,optimizer,criterion,10,scheduler,batch_scheduler=False,best_acc = 0,path = "newtinymodel.pt")

EPOCH: 1 LR: 0.01




Test set: Average loss: 0.0288, Accuracy: 1216/1397 (87.04%)

accuracy increased, Saving model....
EPOCH: 2 LR: 0.01




Test set: Average loss: 0.0134, Accuracy: 1321/1397 (94.56%)

accuracy increased, Saving model....
EPOCH: 3 LR: 0.01




Test set: Average loss: 0.0090, Accuracy: 1345/1397 (96.28%)

accuracy increased, Saving model....
EPOCH: 4 LR: 0.01




Test set: Average loss: 0.0075, Accuracy: 1352/1397 (96.78%)

accuracy increased, Saving model....
EPOCH: 5 LR: 0.01


...



Test set: Average loss: 0.0063, Accuracy: 1356/1397 (97.07%)

accuracy increased, Saving model....
EPOCH: 7 LR: 0.01




Test set: Average loss: 0.0059, Accuracy: 1363/1397 (97.57%)

accuracy increased, Saving model....
EPOCH: 8 LR: 0.01




Test set: Average loss: 0.0058, Accuracy: 1360/1397 (97.35%)

EPOCH: 9 LR: 0.01




Test set: Average loss: 0.0059, Accuracy: 1357/1397 (97.14%)

EPOCH: 10 LR: 0.01




Test set: Average loss: 0.0057, Accuracy: 1361/1397 (97.42%)



In [18]:
train_model(model,device,trainloader,testloader,optimizer,criterion,5,scheduler,batch_scheduler=False,best_acc = 97.4,path = "newtinymodel.pt")

EPOCH: 1 LR: 0.001




Test set: Average loss: 0.0053, Accuracy: 1365/1397 (97.71%)

accuracy increased, Saving model....
EPOCH: 2 LR: 0.001




Test set: Average loss: 0.0054, Accuracy: 1361/1397 (97.42%)

EPOCH: 3 LR: 0.001




Test set: Average loss: 0.0051, Accuracy: 1364/1397 (97.64%)

EPOCH: 4 LR: 0.001




Test set: Average loss: 0.0052, Accuracy: 1362/1397 (97.49%)

EPOCH: 5 LR: 0.001




Test set: Average loss: 0.0053, Accuracy: 1358/1397 (97.21%)



In [20]:
train_model(model,device,trainloader,testloader,optimizer,criterion,5,scheduler,batch_scheduler=False,best_acc = 76.5,path = "newtinymodel.pt")

EPOCH: 1 LR: 0.001




Test set: Average loss: 0.0053, Accuracy: 1359/1397 (97.28%)

accuracy increased, Saving model....
EPOCH: 2 LR: 0.001




Test set: Average loss: 0.0051, Accuracy: 1365/1397 (97.71%)

accuracy increased, Saving model....
EPOCH: 3 LR: 0.001




Test set: Average loss: 0.0050, Accuracy: 1362/1397 (97.49%)

EPOCH: 4 LR: 0.001




Test set: Average loss: 0.0054, Accuracy: 1357/1397 (97.14%)

EPOCH: 5 LR: 0.001




Test set: Average loss: 0.0053, Accuracy: 1362/1397 (97.49%)



In [21]:
torch.save(model, '/content/drive/My Drive/EV4P2/session4/lfw_model_modified.pt')

In [22]:
traced_model = torch.jit.trace(model.to('cpu'), torch.randn(1, 3, 224, 224))

In [23]:
traced_model.save('/content/drive/My Drive/EV4P2/session4/lfw_model_traced_modified.pt')

In [19]:
len(dataset)

4655

In [24]:
dataset.classes

['APJ_Abdul_Kalam',
 'Abdullah_Gul',
 'Adrien_Brody',
 'Alejandro_Toledo',
 'Alvaro_Uribe',
 'Amelie_Mauresmo',
 'Andre_Agassi',
 'Andy_Roddick',
 'Angelina_Jolie',
 'Ann_Veneman',
 'Anna_Kournikova',
 'Ari_Fleischer',
 'Ariel_Sharon',
 'Arnold_Schwarzenegger',
 'Atal_Bihari_Vajpayee',
 'Barack_Obama',
 'Bill_Clinton',
 'Bill_Frist',
 'Bill_Gates',
 'Bill_Graham',
 'Bill_McBride',
 'Bill_Simon',
 'Britney_Spears',
 'Carlos_Menem',
 'Carlos_Moya',
 'Catherine_Zeta-Jones',
 'Chandler_Bing',
 'Charles_Moose',
 'Charles_Taylor',
 'Colin_Farrell',
 'Colin_Powell',
 'Condoleezza_Rice',
 'David_Beckham',
 'David_Nalbandian',
 'Dick_Cheney',
 'Dominique_de_Villepin',
 'Donald_Rumsfeld',
 'Edmund_Stoiber',
 'Eduardo_Duhalde',
 'Elon_Musk',
 'Fernando_Gonzalez',
 'Fidel_Castro',
 'George_Clooney',
 'George_HW_Bush',
 'George_Robertson',
 'George_W_Bush',
 'Gerhard_Schroeder',
 'Gloria_Macapagal_Arroyo',
 'Gonzalo_Sanchez_de_Lozada',
 'Gordon_Brown',
 'Gray_Davis',
 'Guillermo_Coria',
 'Halle_Ber

In [29]:
dataset.classes.index('APJ_Abdul_Kalam')

0

In [41]:
dataset.classes.index('Barack_Obama')

15

In [31]:
dataset.classes.index('Chandler_Bing')

26

In [40]:
dataset.classes.index('Elon_Musk')

39

In [33]:
dataset.classes.index('Joey_Tribianni')

89

In [34]:
dataset.classes.index('Michelle_Obama')

127

In [39]:
dataset.classes.index('Monica_Gellar')

131

In [36]:
dataset.classes.index('Pheobe_Buffay')

146

In [37]:
dataset.classes.index('Rachel_Green')

149

In [38]:
dataset.classes.index('Ross_Gellar')

160

In [42]:
!pip install onnx onnxruntime

     |████████████████████████████████| 7.4MB 7.1MB/s 
     |████████████████████████████████| 4.4MB 40.9MB/s 


In [43]:
import numpy as np
import torch.onnx
import copy
new_model = copy.deepcopy(model)
# Input to the model
torch_model = new_model
batch_size = 1
x = torch.randn(batch_size, 3, 224, 224, requires_grad=True)
torch_out = torch_model(x)

# Export the model
torch.onnx.export(torch_model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "face_recognisation_2.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable lenght axes
                                'output' : {0 : 'batch_size'}})

In [44]:
from onnxruntime.quantization import quantize
from onnxruntime.quantization import QuantizationMode
import onnx

onnx_model = onnx.load("face_recognisation_2.onnx")

quantized_model = quantize(onnx_model, quantization_mode=QuantizationMode.IntegerOps, static=False)

onnx.save(quantized_model, 'face_recognisation_2-quantized.onnx')

In [46]:
print('model size:', os.path.getsize('/content/face_recognisation_2.onnx')/1e+6)
print('quantized model size:', os.path.getsize('/content/face_recognisation_2.onnx')/1e+6)

model size: 94.52891
quantized model size: 94.52891


In [47]:
onnx.save(quantized_model, '/content/drive/My Drive/EV4P2/session4/face_recognisation_2-quantized.onnx')

In [48]:
torch.onnx.export(torch_model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "/content/drive/My Drive/EV4P2/session4/face_recognisation_2.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable lenght axes
                                'output' : {0 : 'batch_size'}})